<a href="https://colab.research.google.com/github/adautofbn/ri_labs/blob/master/lab05/expansao_consulta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pnd
import numpy as npy
import nltk
import re
import collections
import heapq
import time
import bisect
import math
import random
from nltk.tokenize import RegexpTokenizer

nltk.download('stopwords')
resultado = pnd.read_csv("results.csv")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rubens.sousa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
tokenizer = RegexpTokenizer(r'([A-Za-zÁáÉéÍíÓóÚúÃãÕõÇçÂâÊê]{3,27})')
stopwords = nltk.corpus.stopwords.words('portuguese') 
n = 0
rank = {}

for texto in resultado.text:
  palavras = [palavra for palavra in tokenizer.tokenize(texto.lower())
           if not bool(re.search(r'\d', palavra)) and palavra not in stopwords and len(palavra) >= 3]  
  n += 1
  for t in palavras:
    if t not in rank.keys():
      rank[t] = []
    rank[t].append(n)
    
for elemento in rank.items():
  d = dict(collections.Counter(elemento[1]))
  rank[elemento[0]] = list(d.items())
  

In [3]:
consultas = ["presidente", "governo", "país", "pessoas", "mulheres"]

### 1. Calcule as top-10 palavras mais associadas a cada uma dessas 5 palavras de acordo com as 4 métricas que vimos na aula. Você deve produzir uma tabela similar à tabela 6.3 do capítulo 6 do livro texto (pág. 204). Qual métrica você acha que obteve os melhores resultados? Por que?

In [4]:
def n_documentos_ambas(a, b):
  documentos_a = [documentos for documentos,score in rank[a]]
  documentos_b = [documentos for documentos,score in rank[b]]
  n = 0
  
  for documentos in documentos_a:
    if documentos in documentos_b:
      n += 1
      
  return n
  
def n_documentos_palavra(palavra):
  return len(rank[palavra])
  
def n_documentos():
  return resultado.text.count()

In [5]:
def calcula_EMIM(a, b):
  n_a = n_documentos_palavra(a)
  n_b = n_documentos_palavra(b)
  n_ab = n_documentos_ambas(a, b)
  n = n_documentos()
  exp = n * (n_ab / (n_a * n_b))
  emim = 0
  if exp != 0:
    emim = n_ab * npy.log(exp)
  return (emim, b)
  
def calcula_X(a, b):
  n_a = n_documentos_palavra(a)
  n_b = n_documentos_palavra(b)
  n_ab = n_documentos_ambas(a, b)
  n = n_documentos()
  x = (n_ab - (1 / n) * n_a * n_b) ** 2 / (n_a * n_b)
  return (x,b)

def calcula_MIM(a, b):
  n_a = n_documentos_palavra(a)
  n_b = n_documentos_palavra(b)
  n_ab = n_documentos_ambas(a,b)
  mim = n_ab / (n_a * n_b)
  return (mim,b)
  
def calcula_DICE(a, b):
  n_a = n_documentos_palavra(a)
  n_b = n_documentos_palavra(b)
  n_ab = n_documentos_ambas(a, b)
  dice = n_ab / (n_a + n_b)
  return (dice,b)
  

In [6]:
def cria_tabela_de_metricas(query):
  tabela_top_10 = pnd.DataFrame()
  MIM = []
  EMIM = []
  X = []
  Dice = []
  
  for palavra in rank:
    if palavra != query:
      bisect.insort(MIM, calcula_MIM(query,palavra))
      bisect.insort(EMIM,calcula_EMIM(query, palavra))
      bisect.insort(X,calcula_X(query, palavra))
      bisect.insort(Dice,calcula_DICE(query, palavra))

  MIM.reverse()
  EMIM.reverse()
  X.reverse()
  Dice.reverse()
  
  tabela_top_10['MIM'] = [palavra[1] for palavra in MIM]
  tabela_top_10['EMIM'] = [palavra[1] for palavra in EMIM]
  tabela_top_10['X²'] = [palavra[1] for palavra in X]
  tabela_top_10['Dice'] = [palavra[1] for palavra in Dice]
  
  return tabela_top_10.head(10)

In [7]:
tabela_01 = cria_tabela_de_metricas(consultas[0])
tabela_02 = cria_tabela_de_metricas(consultas[1])
tabela_03 = cria_tabela_de_metricas(consultas[2])
tabela_04 = cria_tabela_de_metricas(consultas[3])
tabela_05 = cria_tabela_de_metricas(consultas[4])

####Tabela para a consulta: 'presidente'

In [8]:
tabela_01.index += 1
tabela_01

,MIM,EMIM,X²,Dice
1,útil,bolsonaro,técnicos,bolsonaro
2,úteis,disse,consumidor,disse
3,último,jair,agência,brasil
4,última,lula,vem,jair
5,ótimo,prisão,governos,governo
6,ótima,silva,repasse,feira
7,órgão,livre,necessários,nesta
8,ódio,dias,informa,contra
9,índios,inácio,funciona,ser
10,índio,república,famílias,sobre


####Tabela para a consulta: 'governo'

In [9]:
tabela_02.index += 1
tabela_02

,MIM,EMIM,X²,Dice
1,única,bolsonaro,sendo,bolsonaro
2,último,afirmou,lado,presidente
3,ótimo,ainda,desenvolvimento,brasil
4,ótima,sendo,vou,jair
5,órgão,lado,chefe,disse
6,índios,jair,povo,afirmou
7,étnica,povo,sim,paulo
8,ética,paulo,ainda,ainda
9,época,país,secretário,país
10,ânimo,vai,publicada,ser


####Tabela para a consulta: 'país'

In [10]:
tabela_03.index += 1
tabela_03

,MIM,EMIM,X²,Dice
1,útil,brasil,setor,brasil
2,étnicas,mundo,méxico,governo
3,ética,países,ato,nacional
4,árbitro,nacional,saída,paulo
5,álvaro,setor,importância,mundo
6,álcool,grande,declarou,grande
7,áfrica,ato,mundo,pessoas
8,zelândia,fernando,visita,bolsonaro
9,yeend,eua,oposição,países
10,yad,méxico,países,afirmou


####Tabela para a consulta: 'pessoas'

In [11]:
tabela_04.index += 1
tabela_04

,MIM,EMIM,X²,Dice
1,ótimo,então,então,então
2,ótima,todo,seguinte,país
3,ídolo,deputado,facebook,paulo
4,étnica,país,veio,todo
5,ética,antes,retorno,deputado
6,ânimos,seguinte,procurando,federal
7,âmago,facebook,parece,dias
8,árbitro,realidade,hora,disse
9,álvaro,dentro,homens,onde
10,álcool,sempre,escrito,apenas


####Tabela para a consulta: 'mulheres'

In [12]:
tabela_05.index += 1
tabela_05

,MIM,EMIM,X²,Dice
1,volume,seguro,seguro,seguro
2,verdadeiramente,forte,forte,forte
3,velhice,economista,economista,economista
4,variáveis,aposentadoria,aposentadoria,aposentadoria
5,trouxeram,igual,volume,igual
6,tramitação,homens,verdadeiramente,homens
7,timing,aprovada,velhice,aprovada
8,tensões,problema,variáveis,volume
9,tecnicamente,banco,trouxeram,verdadeiramente
10,tecido,porque,tramitação,velhice


### 2. De acordo com a métrica que deu os melhores resultados na sua opinião, execute agora cada consulta (usando a abordagem documento- ou termo-por-vez)  expandido-a com: os top-3, top-5 e top-10 documentos. O que acontece com a precisão dos resultados em cada caso? Aumenta ou diminui? Justifique bem sua resposta.